#Adding a PowerBI DataSource ID on to a PowerBI DataSet table via PowerBI REST API

In [ ]:
#Read JSON on local

import pandas as pd
import polars as pl

pbi_dataset_df = pd.read_json(r'C:\Users\aserlovsky\Downloads\{filename}.json')
print(pbi_dataset_df)

In [ ]:
#Flatten JSON File function

def flatten_nested_json_df(df):
    
    df = df.reset_index()
    
    print(f"original shape: {df.shape}")
    print(f"original columns: {df.columns}")
    
    
    # search for columns to explode/flatten
    s = (df.applymap(type) == list).all()
    list_columns = s[s].index.tolist()
    
    s = (df.applymap(type) == dict).all()
    dict_columns = s[s].index.tolist()
    
    print(f"lists: {list_columns}, dicts: {dict_columns}")
    while len(list_columns) > 0 or len(dict_columns) > 0:
        new_columns = []
        
        for col in dict_columns:
            print(f"flattening: {col}")
            # explode dictionaries horizontally, adding new columns
            horiz_exploded = pd.json_normalize(df[col]).add_prefix(f'{col}.')
            horiz_exploded.index = df.index
            df = pd.concat([df, horiz_exploded], axis=1).drop(columns=[col])
            new_columns.extend(horiz_exploded.columns) # inplace
        
        for col in list_columns:
            print(f"exploding: {col}")
            # explode lists vertically, adding new columns
            df = df.drop(columns=[col]).join(df[col].explode().to_frame())
            # Prevent combinatorial explosion when multiple
            # cols have lists or lists of lists
            df = df.reset_index(drop=True)
            new_columns.append(col)
        
        # check if there are still dict o list fields to flatten
        s = (df[new_columns].applymap(type) == list).all()
        list_columns = s[s].index.tolist()

        s = (df[new_columns].applymap(type) == dict).all()
        dict_columns = s[s].index.tolist()
        
        print(f"lists: {list_columns}, dicts: {dict_columns}")
        
    print(f"final shape: {df.shape}")
    print(f"final columns: {df.columns}")
    return df

In [ ]:
#Call flatten function
flattened_dataset_df = flatten_nested_json_df(pbi_dataset_df)

In [ ]:
# Iterate per each datasetid per each API call
import requests
import json

list_of_datasourcesbydataset = []
count = 0

dataset_ids = flattened_dataset_df['value.id']
for dataset_id in dataset_ids:
    url = f"https://api.powerbi.com/v1.0/myorg/admin/datasets/{dataset_id}/datasources"

    payload = {}
    headers = {
      'Authorization': 'Bearer {BearerToken}'   }

    response = requests.request("GET", url, headers=headers, data=payload)
    response_dict = json.loads(response.text)
    response_dict['datasetid'] = dataset_id
    list_of_datasourcesbydataset.append(response_dict)
    count += 1
    print(count, dataset_id)
    
print(list_of_datasourcesbydataset)
    

In [ ]:
#Save File
import os
def write_json(new_data, filename=None):
    desired_dir = "C:\\Users\\aserlovsky\\"
    full_path = os.path.join(desired_dir, filename)
    with open(full_path, 'w') as f:
        json_string=json.dumps(new_data)
        f.write(json_string)
        
write_json(list_of_datasourcesbydataset, filename="datasources.json")
# json_string = json.dumps(list_of_datasourcesbydataset)
# print(json_string)



#References

In [ ]:
#Check current working directory (cwd)
import os
cwd = os.getcwd()
print(cwd)

In [ ]:
#POST Access Token
import requests

url = "https://login.microsoftonline.com/{tenantID}/oauth2/token"

payload = 'tenant_id={tenantID}&grant_type=client_credentials&client_id={ClientID}&client_secret={ClientSecret}&resource=https%3A%2F%2Fanalysis.windows.net%2Fpowerbi%2Fapi&scope=api%3A%2F%2F{ClientID}%2FPBIAPI'
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Authorization': 'Bearer {BearerToken}
  'Cookie': {Cookie} #Optional
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

In [ ]:
#API Call to get dataset

import requests
import json

url = f"https://api.powerbi.com/v1.0/myorg/admin/datasets"

payload = {}
headers = {
  'Authorization': 'Bearer {BearerToken}}

response = requests.request("GET", url, headers=headers, data=payload)
response_dict = json.loads(response.text)
print(response_dict)

In [ ]:
#API Call to get datasource

import requests
import json

url = f"https://api.powerbi.com/v1.0/myorg/admin/datasets/{DatasetID}/datasources"

payload = {}
headers = {
  'Authorization': 'Bearer {BearerToken}}

response = requests.request("GET", url, headers=headers, data=payload)
response_dict = json.loads(response.text)
print(response_dict)
